In [ ]:
!pip install -q pyspark

!wget -q 'https://drive.google.com/uc?export=download&id=1O8tTnERl9fJ8b0saO86oYjHXmFL2C8QW' -O 'matrixaddition-W25.txt'
!wget -q 'https://drive.google.com/uc?export=download&id=1s7W2YPARk4ZseL5w0FtZ4otbfULODsMg' -O 'matrixmultiplication-W25.txt'


In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

conf = SparkConf().setAppName('matrixadditionNmultiplication')
sc = SparkContext.getOrCreate(conf=conf)

spark = SparkSession.builder\
    .master("local")\
    .appName("Colab")\
    .config('spark.ui.port', '4050')\
    .getOrCreate()

In [ ]:
def parserFun(line):
    s = line.strip()
    if not s:
        return None
    for ch in [",", ";", "|", "\t"]:
        s = s.replace(ch, " ")
    parts = [p for p in s.split() if p]
    if len(parts) != 4:
        return None
    tag = parts[0].upper()
    try:
        a = int(parts[1]); b = int(parts[2]); v = float(parts[3])
    except:
        return None
    if tag == "M": return ("M", a, b, v)
    if tag == "N": return ("N", a, b, v)
    return None
def fnum(x):
    return str(int(x)) if float(x).is_integer() else str(x)
def txtSave(rdd, out_dir):
    import shutil, os
    if os.path.exists(out_dir):
        shutil.rmtree(out_dir)
    rdd.coalesce(1, shuffle=False).saveAsTextFile(out_dir)

In [ ]:
#part 1 and 2
def matrixAddition(inputpath, opdir):
    ln  = sc.textFile(inputpath)
    par = ln.map(parserFun).filter(lambda x: x is not None)
    crd = par.map(lambda t: ((t[1], t[2]), t[3]))
    sum = crd.reduceByKey(lambda a, b: a + b)
    od = sum.sortByKey().map(lambda kv: f"{kv[0][0]} {kv[0][1]} {fnum(kv[1])}")
    txtSave(od, opdir)
    return od
ad = matrixAddition("matrixaddition-W25.txt", "outPutForAddition")
adp = ad.take(10)
for i, row in enumerate(adp):
    print(f"  {i}: {row}")

  0: 0 0 554
  1: 0 1 -349
  2: 0 2 -1764
  3: 0 3 1483
  4: 0 4 -810
  5: 0 5 1315
  6: 0 6 282
  7: 0 7 -1209
  8: 0 8 -1032
  9: 0 9 -1379


In [ ]:
#part 3 and 4
def matrixmultiplication(inputpath, opdir):
    ln = sc.textFile(inputpath)
    par = ln.map(parserFun).filter(lambda x: x is not None)
    M = par.filter(lambda t: t[0] == "M").map(lambda t: (t[1], t[2], t[3]))  # (i,j,v)
    N = par.filter(lambda t: t[0] == "N").map(lambda t: (t[1], t[2], t[3]))  # (j,k,v)
    I = (M.map(lambda x: x[0]).max() + 1) if not M.isEmpty() else 0
    K = (N.map(lambda x: x[1]).max() + 1) if not N.isEmpty() else 0
    Me = M.flatMap(lambda ijv: [((ijv[0], k), ('M', ijv[1], ijv[2])) for k in range(K)])
    Ne = N.flatMap(lambda jkv: [((i, jkv[1]), ('N', jkv[0], jkv[2])) for i in range(I)])
    grp = Me.union(Ne).groupByKey()
    pr = grp.mapValues(lambda vals: (
        (lambda md, nd: sum(md.get(j,0.0)*nd.get(j,0.0) for j in md if j in nd))(
            {j:v for tag,j,v in vals if tag=='M'},
            {j:v for tag,j,v in vals if tag=='N'}
        )
    ))
    md = pr.sortByKey().map(lambda kv: f"{kv[0][0]} {kv[0][1]} {fnum(kv[1])}")
    txtSave(md, opdir)
    return md
sd = matrixmultiplication("matrixmultiplication-W25.txt", "outputForMultiplication")
mulp = sd.take(10)
for i, row in enumerate(mulp):
    print(f"  {i}: {row}")

  0: 0 0 -8072761
  1: 0 1 -5830281
  2: 0 2 6154135
  3: 0 3 -4741336
  4: 0 4 -413409
  5: 0 5 -2898398
  6: 0 6 -2532699
  7: 0 7 -7421998
  8: 0 8 -1465241
  9: 0 9 -1857565
